# 7. 그림 4-14에서 나머지 8개 화소 값을 게산하시오.

In [1]:
import numpy as np

In [46]:
R = np.array([[1,1,1],[2,1,3],[0,1,0]])
G = np.array([[2,2,2],[1,0,1],[0,0,1]])
B = np.array([[0,3,0],[1,0,1],[1,0,0]])
image = np.array([R,G,B])

c1 = np.array([[0,0,0],[0,0,1],[0,1,0]])
c2 = np.array([[0,2,0],[0,2,0],[0,2,0]])
c3 = np.array([[1,0,0],[0,2,0],[0,0,1]])
kernel = np.array([c1,c2,c3])

padding=1 # zero padding을 옆으로 얼마까지
strides=1

In [52]:
image

array([[[1, 1, 1],
        [2, 1, 3],
        [0, 1, 0]],

       [[2, 2, 2],
        [1, 0, 1],
        [0, 0, 1]],

       [[0, 3, 0],
        [1, 0, 1],
        [1, 0, 0]]])

In [47]:
image_channel, image_height, image_width = image.shape[0], image.shape[1], image.shape[2]
kernel_channel, kernel_height, kernel_width = kernel.shape[0], kernel.shape[1], kernel.shape[2]

In [48]:
output_height = int(((image_height - kernel_height + 2 * padding) / strides) + 1)
output_width= int(((image_width - kernel_width + 2 * padding) / strides) + 1)
output_channel = len(kernel)

In [49]:
output = np.zeros((output_channel, output_height, output_width))
output

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [50]:
if padding != 0:
    imagePadded = np.zeros((image_channel, image_height + padding * 2, image_width + padding * 2))
    imagePadded[:, padding:(-1*padding), padding:(-1*padding)] = image

In [55]:
imagePadded

array([[[0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 2., 1., 3., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 2., 2., 2., 0.],
        [0., 1., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 3., 0., 0.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

In [56]:
for z in range(0, output_channel):
    output_per_channel = np.zeros((output_height, output_width))

    for y in range(0, output_height):
        if (y*strides + kernel_height) <= imagePadded.shape[1]:

            for x in range(0, output_width):                
                if (x*strides + kernel_width) <= imagePadded.shape[2]:
                    output_per_channel[y][x] = np.sum(imagePadded[:,
                                                           y*strides : y*strides + kernel_height,
                                                           x*strides : x*strides + kernel_width] * kernel).astype(np.float32)
    output[z, :, :] = output_per_channel


In [102]:
output[0]

array([[ 9., 13.,  9.],
       [ 9.,  8., 13.],
       [ 5.,  1.,  4.]])

# 8 그림 4-8(B)에서 커널 = np.array([[-1,-1,-1],[0,0,0],[1,1,1]])을 적용한 결과를 쓰시오 이때 덧대기를 0으로 하고 바이어스를 0.5 사용하시오.

In [147]:
import numpy as np

Kernel = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
matrix= np.array([[2,2,2,2,2,1,1,1],
                  [2,2,2,2,2,1,1,1],
                  [2,2,2,2,2,1,1,1],
                  [2,2,2,2,2,1,1,1],
                  [2,2,2,9,9,9,9,9],
                  [2,2,2,9,9,9,9,9],
                  [2,2,2,9,9,9,9,9],
                  [2,2,2,9,9,9,9,9]])


def conv_2d(inputmat, kernel, bias, stride,padding):
    kernel_height, kernel_width = kernel.shape
    input_height, input_width = inputmat.shape
    x = ((input_height - kernel_height + 2*padding) // stride) + 1
    y = ((input_width - kernel_width+ 2*padding) // stride) + 1
    res = np.zeros((x, y))
    
    padding_input = np.pad(inputmat, ((padding, padding), (padding, padding)),
                       'constant', constant_values=0)

    for x1 in range(0, x):
        for y1 in range(0, y):
            input_slice = padding_input[x1 * stride: x1 * stride + kernel_height, y1 * stride: y1 * stride + kernel_width]
            res[x1, y1] = np.sum(input_slice * kernel) + bias

    return res

bias = 2
print(conv_2d(matrix, Kernel, bias, 1,0)) ## 교재 값. 패딩X, bias = 2, 보폭 =1 

bias = 0.5
print(conv_2d(matrix,Kernel,bias,1,1)) ## 패딩 0으로 1 depth, bias = 0.5, 보폭 =1



[[ 2.  2.  2. -1. -1.  2.]
 [ 2.  2.  2. -1. -1.  2.]
 [ 2.  9.  9.  0.  0.  2.]
 [ 2. 16. 16.  1.  1.  2.]
 [ 2. 23. 23.  2.  2.  2.]
 [ 2. 23. 23.  2.  2.  2.]]
[[  4.5   0.5   0.5   0.5  -1.5  -1.5   0.5  -1.5]
 [  6.5   0.5   0.5   0.5  -2.5  -2.5   0.5  -2.5]
 [  6.5   0.5   0.5   0.5  -2.5  -2.5   0.5  -2.5]
 [  6.5   0.5   7.5   7.5  -1.5  -1.5   0.5 -10.5]
 [  6.5   0.5  14.5  14.5  -0.5  -0.5   0.5 -18.5]
 [  6.5   0.5  21.5  21.5   0.5   0.5   0.5 -26.5]
 [  6.5   0.5  21.5  21.5   0.5   0.5   0.5 -26.5]
 [  4.5   0.5  14.5  14.5   0.5   0.5   0.5 -17.5]]


# 9.문제 8에서 보폭을 2로 사용했을 때 결과

In [146]:
import numpy as np

Kernel = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
matrix= np.array([[2,2,2,2,2,1,1,1],
                  [2,2,2,2,2,1,1,1],
                  [2,2,2,2,2,1,1,1],
                  [2,2,2,2,2,1,1,1],
                  [2,2,2,9,9,9,9,9],
                  [2,2,2,9,9,9,9,9],
                  [2,2,2,9,9,9,9,9],
                  [2,2,2,9,9,9,9,9]])


def conv_2d(inputmat, kernel, bias, stride,padding):
    kernel_height, kernel_width = kernel.shape
    input_height, input_width = inputmat.shape
    x = ((input_height - kernel_height + 2*padding) // stride) + 1
    y = ((input_width - kernel_width+ 2*padding) // stride) + 1
    res = np.zeros((x, y))
    
    padding_input = np.pad(inputmat, ((padding, padding), (padding, padding)),
                       'constant', constant_values=0)

    for x1 in range(0, x):
        for y1 in range(0, y):
            input_slice = padding_input[x1 * stride: x1 * stride + kernel_height, y1 * stride: y1 * stride + kernel_width]
            res[x1, y1] = np.sum(input_slice * kernel) + bias

    return res

bias = 0.5
print(conv_2d(matrix,Kernel,bias,2,1)) ## 패딩 0으로 1 depth, bias = 0.5, 보폭 =2



[[ 4.5  0.5 -1.5  0.5]
 [ 6.5  0.5 -2.5  0.5]
 [ 6.5 14.5 -0.5  0.5]
 [ 6.5 21.5  0.5  0.5]]
